# Importando Bibliotecas

In [9]:
import pandas as pd
import seaborn as srn
import statistics as sts

# Importando Dados

In [10]:
dataset = pd.read_csv("Churn.csv", sep=";")
dataset.head()
dataset.shape


(999, 12)

# Primeiro problema é dar nomes as colunas


In [13]:
dataset.columns = ["Id","Score","Estado","Genero","Idade","Patrimonio","Saldo",
                   "Produtos","TemCartCredito",
                    "Ativo","Salario","Saiu"]

# Visulizar

In [14]:
dataset.head()

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
0,1,619,RS,Feminino,42,2,0,1,1,1,10134888.0,1
1,2,608,SC,Feminino,41,1,8380786,1,0,1,11254258.0,0
2,3,502,RS,Feminino,42,8,1596608,3,1,0,11393157.0,1
3,4,699,RS,Feminino,39,1,0,2,0,0,9382663.0,0
4,5,850,SC,Feminino,43,2,12551082,1,1,1,790841.0,0


# Explorar dados categoricos


In [ ]:
#Estado
agrupado = dataset.groupby(['Estado']).size()
agrupado
agrupado.plot.bar(color='gray')

In [ ]:
#Genero
agrupado = dataset.groupby(['Genero']).size()
agrupado
agrupado.plot.bar(color='gray')

In [ ]:
agrupado = dataset.groupby(['Temperatura']).size()
agrupado
dataset.loc[dataset['Temperatura'] < -100 or dataset['Temperatura'] > 100]
#substituir valores
mediana = sts.median(dataset['Temperatura'])
mediana
dataset.loc[dataset['Temperatura'] < -100 or dataset['Temperatura'] > 100, 'Temperatura'] = mediana

# Explorar colunas numéricas

In [ ]:
#Score
dataset['Score'].describe()
srn.boxplot(dataset['Score']).set_title('Score')
srn.distplot(dataset['Score']).set_title('Score')


In [ ]:
#Idade
dataset['Idade'].describe()
srn.boxplot(dataset['Idade']).set_title('Idade')
srn.distplot(dataset['Idade']).set_title('Idade')

In [ ]:
#Saldo
dataset['Saldo'].describe()
srn.boxplot(dataset['Saldo']).set_title('Saldo')
srn.distplot(dataset['Saldo']).set_title('Saldo')

In [ ]:
#Salário
dataset['Salario'].describe()
srn.boxplot(dataset['Salario']).set_title('Salario')
srn.distplot(dataset['Salario']).set_title('Salario')

# Contamos valores NAN

In [ ]:
#genero e salário
dataset.isnull().sum()

# Remover nas e substiutir pela mediana

In [ ]:
#salarios
dataset['Salario'].describe()
mediana = sts.median(dataset['Salario'])
print(mediana)

In [ ]:
#substituir NAN por mediana
dataset['Salario'].fillna(mediana, inplace=True)

In [ ]:
#Verificamos se NAN não existem mais
dataset['Salario'].isnull().sum()

# Genero, falta de padronização e NAs

In [ ]:
agrupado = dataset.groupby(['Genero']).size()
print(agrupado)

In [ ]:
#total de Nas
dataset['Genero'].isnull().sum()

In [ ]:
#preenche NAs com Masculino (moda)
dataset['Genero'].fillna('Masculino', inplace=True)


In [ ]:
#verificamos novamente NANs
dataset['Genero'].isnull().sum()

# Padroniza de acordo com o dominio

In [ ]:
dataset.loc[dataset['Genero'] == 'M', 'Genero'] = "Masculino"
dataset.loc[dataset['Genero'].isin(['Fem', 'F']), 'Genero'] = "Feminino"

In [ ]:
#visualiza o resultado
agrupado = dataset.groupby(['Genero']).size()
agrupado

# Idades fora do dominio

In [15]:
dataset['Idade'].describe()

count    999.000000
mean      38.902903
std       11.401912
min      -20.000000
25%       32.000000
50%       37.000000
75%       44.000000
max      140.000000
Name: Idade, dtype: float64

In [16]:
#visualizar
dataset.loc[(dataset['Idade'] <  0 )  | ( dataset['Idade'] >  120) ]

,Id,Score,Estado,Genero,Idade,Patrimonio,Saldo,Produtos,TemCartCredito,Ativo,Salario,Saiu
867,869,636,RS,Feminino,-10,1,17083346,1,1,0,11051028.0,1
984,986,773,RS,Masculino,-20,1,12453278,2,0,1,1172357.0,0
990,992,655,RS,Masculino,140,5,93147,2,1,0,6621413.0,0


In [17]:
#calular a mediana
mediana = sts.median(dataset['Idade'])
print(mediana)

37


In [18]:
#substituir
dataset.loc[(dataset['Idade'] < 0) | (dataset['Idade'] > 120), 'Idade'] = mediana

In [ ]:
#verificamos se ainda existem idades fora do domínio
dataset.loc[(dataset['Idade'] <  0 )  | ( dataset['Idade'] >  120) ]

#Dados duplicados, buscamos pelo ID

In [ ]:
dataset[dataset.duplicated(['Id'],keep=False)]

In [ ]:
#excluimso pelo ID
dataset.drop_duplicates(subset="Id", keep='first',inplace=True)
#buscamos duplicados 
dataset[dataset.duplicated(['Id'],keep=False)]

#Estado foram do domínio

In [ ]:
agrupado = dataset.groupby(['Estado']).size()
agrupado

In [ ]:
#atribuomos RS (moda)
dataset.loc[dataset['Estado'].isin( ['RP','SP','TD']), 'Estado'] = "RS"
agrupado = dataset.groupby(['Estado']).size()

In [ ]:
#verificamos o resultado
agrupado

#Outliers em salário, vamos considerar 2 desvios padrão

In [ ]:
desv = sts.stdev(dataset['Salario'])
desv

In [ ]:
#definir padrão como maior que 2 desvios padrão
#checamos se algum atende critério
dataset.loc[dataset['Salario'] >= 2 * desv]

In [ ]:
#vamos atualiar salarios para mediana, calculamos
mediana = sts.median(dataset['Salario'])
print(mediana)

In [ ]:
#atribumos
dataset.loc[dataset['Salario'] >=  2 * desv, 'Salario'] = mediana
#checamos se algum atende critério
dataset.loc[dataset['Salario'] >=  2 * desv ]
dataset.head()
dataset.shape